# Day 17 - Planning and Learning with Tabular Methods

## When the Model Is Wrong

### $Exercise\ \mathcal{8.2}$

#### Why did the Dyna agent with exploration bonus, Dyna-Q+, perform better in the first phase as well as in the second phase of the blocking and shortcut experiments?

Once both agents found the path to the goal, the default Dyna-Q agent would have to keep repeating suboptimal actions until the value estimate of a "neighboring," optimal action is greater. The Dyna-Q+ agent explores these sooner, and therefore finds an ~optimal solution quicker.

### $Exercise\ \mathcal{8.3}$

#### Careful inspection of Figure 8.5 reveals that the di↵erence between Dyna-Q+ and Dyna-Q narrowed slightly over the first part of the experiment. What is the reason for this?

As the Dyna-Q+ agent keeps exploring, it can never reach the optimal policy, which Dyna-Q does. So while Dyna-Q+ finds better solutions faster, Dyna-Q finds the optimal policy at some point.